In [2]:
from typing import Tuple, List
from datetime import datetime, timedelta, timezone
import pytz

import numpy as np

from sgp4.api import Satrec
from sgp4.api import jday
from astropy import coordinates
from astropy.coordinates.builtin_frames import GCRS, ITRS
import plotly.express as px
import plotly.graph_objects as go
import pymap3d as pm

from coordinate_system.EarhRotating import build_default_rotating_cs
from XtoY.cartesianfromgeo import get_cartesian_geo_from_geo_angles
from coordinate_system.earth_models import get_wgs_84
from XtoY.xyz_rfilam import xyz_to_rfilam

from matplotlib import pyplot as plt

In [3]:
NOAA19 = {"name": "NOAA 19", "s": "1 33591U 09005A   21279.05525385  .00000072  00000-0  64475-4 0  9998",
     "t": "2 33591  99.1783 302.4748 0013975 178.4339 181.6878 14.12498974652537"}

In [4]:
rotator = build_default_rotating_cs() #строит матрицу вращения

wgs84 = get_wgs_84() # всемирная система геодезических параметров Земли 1984 года, в число которых входит система геоцентрических координат

In [5]:
def get_satellite_trajectory(tle2lines: Tuple[str, str], time_points: List[datetime]):
    """Получение траектории спутника из  TLE."""
    satellite = Satrec.twoline2rv(*tle2lines)
    jd_fr = np.array([jday(*(point.utctimetuple()[:6])) for point in time_points])
    result = [satellite.sgp4(jd, fr)[1] for jd, fr in jd_fr]
    return np.array(result)

def get_terrestrial_point_coordinates(geo_angles: Tuple[float, float], time_points: List[datetime]):
    """Получение координат спутника в ИСО."""
    coordinates = get_cartesian_geo_from_geo_angles(latitude=geo_angles[0], longitude=geo_angles[1], model=wgs84)
    time_points_timestamps = [time_point.timestamp() for time_point in time_points]
    rotation_matrix_list = rotator.get_matrix(time_points_timestamps)
    coordinates_in_inertial_system = np.array([rm.dot(coordinates) / 1000 for rm in rotation_matrix_list])
    return coordinates_in_inertial_system

def move_satellite_trajectory_to_earth_coordinates(
    trajectory: np.ndarray, time_points: List[datetime]
):
    """Перевод тракетории спутника в земные координаты."""
    time_points_timestamps = [time_point.timestamp() for time_point in time_points]
    rotation_matrix_list = rotator.get_matrix(time_points_timestamps)
    trajectory_in_earth_coordinates = np.array([rm.dot(coordinate) for rm, coordinate in zip(rotation_matrix_list, trajectory)])
    return trajectory_in_earth_coordinates

def get_related_spherical_coordinates(ref_point_coordinate, target_coordinates):
    """Получение сферических координат."""
    dr_list = target_coordinates - ref_point_coordinate
#     print(dr_list)
    e1 = ref_point_coordinate
    e1 = e1 / np.linalg.norm(e1)
    e2 = np.cross([0, 0, 1], ref_point_coordinate)
    e2 = e2 / np.linalg.norm(e2)
    e3 = np.cross(e1, e2)
    e3 = e3 / np.linalg.norm(e3)
#     print(e1, e2, e3)

    coor_e1 = np.dot(dr_list, e1)
    coor_e2 = np.dot(dr_list, e2)
    coor_e3 = np.dot(dr_list, e3)
    length = np.linalg.norm(dr_list, axis=1)
#     print(coor_e1)
    
    lam = np.arcsin(coor_e3 / length) * 180 / np.pi
    phi = -np.arctan(coor_e2 / coor_e1) * 180 / np.pi
    return phi, lam, coor_e1

In [6]:
"""Перевод из НеИСО в AER."""
def ECEF_to_ENU(r_sat):
    x_sat = r_sat[0]
    y_sat = r_sat[1]
    z_sat = r_sat[2]

    lon = np.radians(37.518151) #latitude
    lat = np.radians(55.930148) #longitude
    alt = 250

    x0,y0,z0 = pm.geodetic2ecef(lat,lon,alt)

    r_lk = np.array([x0,y0,z0])


    transition = np.array([[-np.sin(lat),np.cos(lat),0],[-np.cos(lat)*np.sin(lon), -np.sin(lat)*np.sin(lon), np.cos(lon)],\
                           [np.cos(lat)*np.cos(lon), np.sin(lat)*np.cos(lon), np.sin(lon)]])

    #define local coordinate system (ENU) about the receiver
    enu = transition @ (r_sat - r_lk)
    return enu

def ENU_to_AER(enu):
    e = enu[0]
    n = enu[1]
    u = enu[2]
    return ENU_to_Aer_point(e,n,u)

def ENU_to_Aer_point(e,n,u):
    if abs(e) < 1e-3:
        e = 0.0
    if abs(n) < 1e-3:
        n = 0.0
    if abs(u) < 1e-3:
        u = 0.0
    r = np.hypot(e, n)
    slantRange = np.hypot(r, u)
    elev = np.arctan2(u, r)
    az = np.arctan2(e, n) % 6.283185307179586
    return az, elev, slantRange

def ECEF2AER(r_sat):
    return ENU_to_AER(ECEF_to_ENU(r_sat))

In [8]:
t0 = datetime(2021, 10, 6, 8, 6, 1, tzinfo=timezone.utc)  #время начала расчета 
time_points = [t0 + timedelta(seconds=i * 10)  for i in range(2000)]
print(time_points[-1])   

lk_coord = [55.930148 / 180 * np.pi, 37.518151 / 180 * np.pi]   #координаты ЛК

plots = []

name = NOAA19["name"]
s = NOAA19["s"]
t = NOAA19["t"]
trajectory = get_satellite_trajectory((s, t), time_points)
lk_coordinate = get_cartesian_geo_from_geo_angles(latitude=lk_coord[0], longitude=lk_coord[1], model=wgs84) / 1000
earth_sk_trajectory = move_satellite_trajectory_to_earth_coordinates(trajectory, time_points)
#     phis, lams, coor_e1 = get_related_spherical_coordinates(lk_coordinate, earth_sk_trajectory)
coord = pm.ecef2aer(earth_sk_trajectory[:, 0] * 1000, 
                    earth_sk_trajectory[:, 1] * 1000, 
                    earth_sk_trajectory[:, 2] * 1000, 
                    lk_coord[0], lk_coord[1], 130, deg=False)
lams, phis, coor_e1 = coord
lams = np.rad2deg(lams)
phis = np.rad2deg(phis)
phi=[]
lam=[]
times=[]
    
for p, l, c, txt in zip(phis, lams, coor_e1, time_points):
#         if c > 0:
    if p > 0:
        phi.append(p)
        lam.append(l)
        times.append(txt.astimezone(pytz.timezone("Europe/Moscow")))
plots.append(go.Scatterpolar(
        r = phi,
        theta = lam,
        text = times,
        name=name, 
        mode = 'markers'
    ))
    
fig = go.Figure(data=
    plots,
)
fig.update_layout(
    polar = dict(
      radialaxis_range = [90, 0]
    ))
fig.show()

2021-10-06 13:39:11+00:00
